# Vader

In [2]:
# Uncomment and run this cell if the relative imports are not working
import sys
sys.path.append('..')

In [43]:
from utils.db_client import DBClient
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [44]:
db = DBClient(secrets_path = "../configs/db_secrets.json")

I'm trying
Connected to political tweets DB


In [45]:
data_query = """
with random_tweets as (
    select tweet_text_raw, Random() from staging.{}
    where tweet_date between '2018-01-01' and '2019-01-01'
    order by Random()
    limit 100)
select tweet_text_raw
from random_tweets;
"""

In [56]:
tweets = db.read(data_query.format("senate"))
tweet_text = [tweet[0][0] for tweet in tweets]

In [57]:
#nltk.download()
sid = SentimentIntensityAnalyzer()

In [58]:
for tweet in tweets:
    tweet_text = tweet[0]
    print(tweet_text)
    ss = sid.polarity_scores(tweet_text)
    print(ss)

The movement has spread west to Arizona! I stand in #solidarity with you, my fellow teachers.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I fought hard to make sure the Senate Farm Bill included provisions to help support rural broadband, and I’ll keep fighting to help open doors for these hardworking Maine people. #mepolitics
{'neg': 0.157, 'neu': 0.581, 'pos': 0.263, 'compound': 0.6369}
Trump is self-enriching like there’s no tomorrow...and no constitution. #Emoluments https://t.co/el4dMPqL6a
{'neg': 0.295, 'neu': 0.537, 'pos': 0.168, 'compound': -0.2263}
Who does Bob Hugin stand with? America or Russia? Greedy CEO Hugin clearly put Russia first when he skyrocketed drug prices for US consumers, yet handed out steep discounts to Russians. https://t.co/iawPdVPZ06
{'neg': 0.065, 'neu': 0.848, 'pos': 0.087, 'compound': 0.1926}
It’s good to know that there are still some issues that rise above partisan politics – and protecting the International Space Station is one of them. htt